In [1]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
import re


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

ytt_api = YouTubeTranscriptApi()
captions = ytt_api.fetch("yGWJo6a4g4w").snippets

# Assuming `snippets` is your list of FetchedTranscriptSnippet objects
full_text = " ".join([item.text for item in captions])
full_text

"hello and welcome to one minute book review my name is Owen and today I'll be reviewing atomic habit by James clear now we all want to build better habits and in this book James Clare talks about systematic steps that we can take in order to build better habits the author builds upon the book the power of habit by Charles Duhigg and gives us practical tools such as the two-minute rule and habit stacking to give us the necessary things that we need to build better habit what I liked about this book was the practical element of it a lot of these books that talk about habits just talk about theory but this book gives us actionable steps that we can use to actually improve our habits in practice this book is incredibly valuable for anyone wanting to understand how to implement better habits into their life and also for people who are trying to get rid of negative ones if you're new to habit formation I think this is a great book for you to try and understand the key principles that go beh

In [3]:
data = []

for item in captions:
    sentence = item.text
    start_time = item.start
    # combine sentence and start_time into one string and add it to data
    data.append(f"{sentence} ({start_time})")

# join the entire list into a single string
youtube_captions = " ".join(data)
print(youtube_captions)


hello and welcome to one minute book (0.0) review my name is Owen and today I'll be (1.23) reviewing atomic habit by James clear (3.06) now we all want to build better habits (5.43) and in this book James Clare talks about (7.14) systematic steps that we can take in (9.36) order to build better habits the author (12.059) builds upon the book the power of habit (14.04) by Charles Duhigg and gives us practical (15.63) tools such as the two-minute rule and (17.97) habit stacking to give us the necessary (19.98) things that we need to build better (22.199) habit what I liked about this book was (24.18) the practical element of it a lot of (25.859) these books that talk about habits just (27.599) talk about theory but this book gives us (29.369) actionable steps that we can use to (31.65) actually improve our habits in practice (33.899) this book is incredibly valuable for (35.88) anyone wanting to understand how to (37.89) implement better habits into their life (39.989) and also for peopl

In [4]:
# ---------- Imports ----------
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage , HumanMessage ,AIMessage
from dotenv import load_dotenv
load_dotenv()
from langgraph.checkpoint.memory import InMemorySaver
# ---------- LLM Configuration ----------
google = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# ---------- Define Chat State ----------
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# ---------- Define Chat Node ----------
def chat_node(state: ChatState):
    messages = state["messages"]
    response = google.invoke(messages)
    return {"messages": [response]}

# ---------- Checkpointer ----------
checkpointer =InMemorySaver()   # (you can change path if needed)

# ---------- Build Graph ----------
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

# ---------- Compile chatbot ----------
chatbot = graph.compile(checkpointer=checkpointer)
CONFIG = {'configurable': {'thread_id':"newthread"}}

In [5]:
output_dict = {
    "human": [],
    "ai": []
}


In [6]:
user_input = input("Enter your message: ")
if output_dict['human'] == []:
    user_input = f"""You are a helpful assistant.
        Answer ONLY from the provided transcript context.
        If the context is insufficient, just say you don't know.
        {youtube_captions}  Question: {user_input}""" 


In [7]:
result = chatbot.invoke(
                {'messages': [HumanMessage(content=user_input)]},
                config=CONFIG,
)
print(user_input)
for msg in result['messages']:
    if isinstance(msg, AIMessage):
            print(msg.content)

You are a helpful assistant.
        Answer ONLY from the provided transcript context.
        If the context is insufficient, just say you don't know.
        hello and welcome to one minute book (0.0) review my name is Owen and today I'll be (1.23) reviewing atomic habit by James clear (3.06) now we all want to build better habits (5.43) and in this book James Clare talks about (7.14) systematic steps that we can take in (9.36) order to build better habits the author (12.059) builds upon the book the power of habit (14.04) by Charles Duhigg and gives us practical (15.63) tools such as the two-minute rule and (17.97) habit stacking to give us the necessary (19.98) things that we need to build better (22.199) habit what I liked about this book was (24.18) the practical element of it a lot of (25.859) these books that talk about habits just (27.599) talk about theory but this book gives us (29.369) actionable steps that we can use to (31.65) actually improve our habits in practice (33.8

In [8]:
result['messages']

[HumanMessage(content="You are a helpful assistant.\n        Answer ONLY from the provided transcript context.\n        If the context is insufficient, just say you don't know.\n        hello and welcome to one minute book (0.0) review my name is Owen and today I'll be (1.23) reviewing atomic habit by James clear (3.06) now we all want to build better habits (5.43) and in this book James Clare talks about (7.14) systematic steps that we can take in (9.36) order to build better habits the author (12.059) builds upon the book the power of habit (14.04) by Charles Duhigg and gives us practical (15.63) tools such as the two-minute rule and (17.97) habit stacking to give us the necessary (19.98) things that we need to build better (22.199) habit what I liked about this book was (24.18) the practical element of it a lot of (25.859) these books that talk about habits just (27.599) talk about theory but this book gives us (29.369) actionable steps that we can use to (31.65) actually improve ou

In [140]:
from langchain_core.messages import HumanMessage, AIMessage

messages_list = result['messages']

for msg in messages_list:
    if isinstance(msg, HumanMessage):
        output_dict['human'].append(msg.content)
    elif isinstance(msg, AIMessage):
        output_dict['ai'].append(msg.content)


In [141]:
output_dict

{'human': ["You are a helpful assistant.\n        Answer ONLY from the provided transcript context.\n        If the context is insufficient, just say you don't know.\n        hello and welcome to one minute book (0.0) review my name is Owen and today I'll be (1.23) reviewing atomic habit by James clear (3.06) now we all want to build better habits (5.43) and in this book James Clare talks about (7.14) systematic steps that we can take in (9.36) order to build better habits the author (12.059) builds upon the book the power of habit (14.04) by Charles Duhigg and gives us practical (15.63) tools such as the two-minute rule and (17.97) habit stacking to give us the necessary (19.98) things that we need to build better (22.199) habit what I liked about this book was (24.18) the practical element of it a lot of (25.859) these books that talk about habits just (27.599) talk about theory but this book gives us (29.369) actionable steps that we can use to (31.65) actually improve our habits in

In [142]:
output_dict['ai']

['Owen reviews Atomic Habits by James Clear, highlighting its practical tools like the two-minute rule and habit stacking for building better habits.',
 'Owen reviews Atomic Habits by James Clear, highlighting its practical tools like the two-minute rule and habit stacking for building better habits.',
 "You're welcome! Is there anything else I can help you with?"]

# Whole Code


In [1]:
# ---------- Imports ----------
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from dotenv import load_dotenv

load_dotenv()

# ---------- LLM Configuration ----------
google = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# ---------- Define Chat State ----------
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# ---------- Define Chat Node ----------
def chat_node(state: ChatState):
    messages = state["messages"]
    response = google.invoke(messages)
    return {"messages": [response]}

# ---------- Checkpointer ----------
checkpointer = InMemorySaver()   # Use memory to store messages in RAM

# ---------- Build Graph ----------
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

# ---------- Compile chatbot ----------
chatbot = graph.compile(checkpointer=checkpointer)

# ---------- Configuration ----------
CONFIG = {'configurable': {'thread_id': "newthread"}}
output_dict = {
    "human": [],
    "ai": []
}

# ---------- Chat Loop ----------
while True:
    user_input = input("Enter your message: ")
    print("user:" , user_input)
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    if output_dict['human'] == []:
        # First message -> prepend instructions + transcript
        user_input = f"""You are a helpful assistant.
    Answer ONLY from the provided transcript context and also the time of that particular sentence in give in () brackets
    If the context is insufficient, just say you don't know.
    {youtube_captions}
    Question: {user_input}"""

    result = chatbot.invoke(
        {'messages': [HumanMessage(content=user_input)]},
        config=CONFIG,
    )

    # ---------- Save messages ----------
    messages_list = result['messages']

    for msg in messages_list:
        if isinstance(msg, HumanMessage):
            output_dict['human'].append(msg.content) if msg.content not in output_dict['human'] else None
        elif isinstance(msg, AIMessage):
            output_dict['ai'].append(msg.content) if msg.content not in output_dict['ai'] else None 
    print("AI :" ,output_dict['ai'][-1])

user: what is this Video For


NameError: name 'youtube_captions' is not defined